In [ ]:
import numpy as np
import pandas as pd
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score
import plotly.plotly as py
import plotly.graph_objs as go
import plotly.io as pio
from plotly.offline import init_notebook_mode, iplot
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.covariance import EmpiricalCovariance
from sklearn.covariance import GraphicalLasso

from scipy import linalg
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl
from matplotlib import colors

from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
from sklearn.decomposition import PCA
from sklearn import model_selection
from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score, StratifiedKFold, learning_curve, ShuffleSplit
import numpy as np
import pandas as pd
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import roc_auc_score
from sklearn.preprocessing import StandardScaler
from sklearn.feature_selection import VarianceThreshold
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis, LinearDiscriminantAnalysis
from sklearn.pipeline import Pipeline
from tqdm import tqdm_notebook
import warnings
import multiprocessing
from scipy.optimize import minimize  
import time
from sklearn.model_selection import GridSearchCV, train_test_split

from sklearn.mixture import GaussianMixture
import warnings
warnings.simplefilter('ignore')
init_notebook_mode()
sns.set()
%matplotlib inline

In [ ]:
train = pd.read_csv('../input/train.csv')
test = pd.read_csv('../input/test.csv')
cols = [c for c in train.columns if c not in ['id', 'target', 'wheezy-copper-turtle-magic']]

In [ ]:
%%time
#
# QDA (Grid Search) and create oof, preds
#
oof = np.zeros(len(train))
preds = np.zeros(len(test))
params = [{'reg_param': [0.1, 0.2, 0.3, 0.4, 0.5]}]

# 512 models
reg_params = np.zeros(512)
for i in tqdm_notebook(range(512)):

    train2 = train[train['wheezy-copper-turtle-magic']==i]
    test2 = test[test['wheezy-copper-turtle-magic']==i]
    idx1 = train2.index; idx2 = test2.index
    train2.reset_index(drop=True,inplace=True)

    data = pd.concat([pd.DataFrame(train2[cols]), pd.DataFrame(test2[cols])])
    pipe = Pipeline([('vt', VarianceThreshold(threshold=2)), ('scaler', StandardScaler())])
    data2 = pipe.fit_transform(data[cols])
    train3 = data2[:train2.shape[0]]; test3 = data2[train2.shape[0]:]

    skf = StratifiedKFold(n_splits=11, random_state=42)
    for train_index, test_index in skf.split(train2, train2['target']):

        qda = QuadraticDiscriminantAnalysis()
        clf = GridSearchCV(qda, params, cv=4)
        clf.fit(train3[train_index,:],train2.loc[train_index]['target'])
        reg_params[i] = clf.best_params_['reg_param']
        oof[idx1[test_index]] = clf.predict_proba(train3[test_index,:])[:,1]
        preds[idx2] += clf.predict_proba(test3)[:,1] / skf.n_splits


In [ ]:
auc = roc_auc_score(train['target'], oof)
print(f'AUC: {auc:.5}')


In [ ]:
for itr in range(4):
    test['target'] = preds
    test.loc[test['target'] > 0.955, 'target'] = 1 # initial 94
    test.loc[test['target'] < 0.045, 'target'] = 0 # initial 06
    usefull_test = test[(test['target'] == 1) | (test['target'] == 0)]
    new_train = pd.concat([train, usefull_test]).reset_index(drop=True)
    print(usefull_test.shape[0], "Test Records added for iteration : ", itr)
    new_train.loc[oof > 0.995, 'target'] = 1 # initial 98
    new_train.loc[oof < 0.005, 'target'] = 0 # initial 02
    preds = np.zeros(len(test))
    for i in tqdm_notebook(range(512)):

        train2 = new_train[new_train['wheezy-copper-turtle-magic']==i]
        test2 = test[test['wheezy-copper-turtle-magic']==i]
        idx1 = train[train['wheezy-copper-turtle-magic']==i].index
        idx2 = test2.index
        train2.reset_index(drop=True,inplace=True)

        data = pd.concat([pd.DataFrame(train2[cols]), pd.DataFrame(test2[cols])])
        pipe = Pipeline([('vt', VarianceThreshold(threshold=2)), ('scaler', StandardScaler())])
        data2 = pipe.fit_transform(data[cols])
        train3 = data2[:train2.shape[0]]
        test3 = data2[train2.shape[0]:]

        skf = StratifiedKFold(n_splits=11, random_state=time.time)
        for train_index, test_index in skf.split(train2, train2['target']):
            oof_test_index = [t for t in test_index if t < len(idx1)]
            
            clf = QuadraticDiscriminantAnalysis(reg_params[i])
            clf.fit(train3[train_index,:],train2.loc[train_index]['target'])
            if len(oof_test_index) > 0:
                oof[idx1[oof_test_index]] = clf.predict_proba(train3[oof_test_index,:])[:,1]
            preds[idx2] += clf.predict_proba(test3)[:,1] / skf.n_splits
    auc = roc_auc_score(train['target'], oof)
    print(f'AUC: {auc:.5}')
    

In [ ]:
t = test.copy()
t['target'] = preds
t.loc[t['target'] > 0.955, 'target'] = 1 # initial 94
t.loc[t['target'] < 0.045, 'target'] = 0 # initial 06
usefull_test = t[(t['target'] == 1) | (t['target'] == 0)]
new_train = pd.concat([train, usefull_test]).reset_index(drop=True)
print(usefull_test.shape[0], "Test Records added for iteration : ", itr)
new_train.loc[oof > 0.995, 'target'] = 1 # initial 98
new_train.loc[oof < 0.005, 'target'] = 0 # initial 02


In [ ]:
%%time
#
# clustering evaluation
#
random_state=42
CLUSTER_NUM = 3

cols = [c for c in train.columns if c not in ['id', 'target']]
cols.remove('wheezy-copper-turtle-magic')
oof = np.zeros(len(train))
preds = np.zeros(len(test))

# for i in tqdm_notebook([100,101,102]):
for i in tqdm_notebook(range(512)):
    train2 = new_train[new_train['wheezy-copper-turtle-magic']==i]
    # test2 = test[test['wheezy-copper-turtle-magic']==i]
    idx1 = train2.index;
    # idx2 = test2.index
    train2.reset_index(drop=True,inplace=True)

    pipe = Pipeline([('vt', VarianceThreshold(threshold=2)), ('scaler', StandardScaler())])
    # data = pd.concat([pd.DataFrame(train2[cols]), pd.DataFrame(test2[cols])])
    # data2 = pipe.fit_transform(data[cols])
    # train3 = data2[:train2.shape[0]]; test3 = data2[train2.shape[0]:]

    
    #
    # ここから target ごとに pipe したものから 教師なし分類a
    #
    sel = VarianceThreshold(threshold=1.5).fit(train2[cols])

    X_train_0_df = train2[train2['target'] == 0]
    X_train_1_df = train2[train2['target'] == 1]
    X_train_0 = sel.transform(X_train_0_df[cols])
    X_train_1 = sel.transform(X_train_1_df[cols])


    # ----- X_train_0 -----
    var_r_list = []
    for r in range(42,52):
        gm = GaussianMixture(n_components=CLUSTER_NUM,
                             init_params='kmeans',
                             covariance_type='full', 
                             max_iter=100,
                             n_init=1,
                             random_state=r,
                        )
        gm.fit(X_train_0)
        X_train_0_df['cluster'] = gm.predict(X_train_0)
        var = X_train_0_df.cluster.value_counts().var()
        var_r_list.append((var, r))

    _, r = min(var_r_list)
    gm = GaussianMixture(n_components=CLUSTER_NUM,
                         init_params='kmeans',
                         covariance_type='full', 
                         max_iter=100,
                         n_init=1,
                         random_state=r,
                        )
    gm.fit(X_train_0)
    X_train_0_df['cluster'] = gm.predict(X_train_0)
    means0 = gm.means_
    co0 = gm.covariances_

    # ----- X_train_1 -----
    var_r_list = []
    for r in range(42,52):
        gm = GaussianMixture(n_components=CLUSTER_NUM,
                             init_params='kmeans',
                             covariance_type='full', 
                             max_iter=100,
                             n_init=1,
                             random_state=r,
                        )
        gm.fit(X_train_1)
        X_train_1_df['cluster'] = gm.predict(X_train_1)
        var = X_train_1_df.cluster.value_counts().var()
        var_r_list.append((var, r))

    _, r = min(var_r_list)
    gm = GaussianMixture(n_components=CLUSTER_NUM,
                         init_params='kmeans',
                         covariance_type='full', 
                         max_iter=100,
                         n_init=1,
                         random_state=r,
                        )
    gm.fit(X_train_1)
    X_train_1_df['cluster'] = CLUSTER_NUM + gm.predict(X_train_1)
    means1 = gm.means_
    co1 = gm.covariances_
    
    # ----- concat -----

    X_train_with_c_df = pd.concat([X_train_0_df, X_train_1_df])
    means01 = np.concatenate((means0,means1))
    co01 = np.concatenate((co0,co1))
    
    # plt.figure(figsize=(16,6))
    # plt.subplot(1,2,1)
    # plt.hist(X_train_with_c_df.cluster)
    # plt.title('cluster distribution')

    
    #
    # ----- gmm fitting for all data !!!! 
    #
    gm2 = GaussianMixture(n_components=CLUSTER_NUM * 2,
                          # init_params='random',
                          covariance_type='full', 
                          max_iter=100,
                          n_init=1,
                          means_init=means01,
                          precisions_init=np.linalg.inv(co01),
                         random_state=random_state,
                          )
    # ここから gmm コピペ
    train2 = train[train['wheezy-copper-turtle-magic']==i]
    test2 = test[test['wheezy-copper-turtle-magic']==i]
    idx1 = train2.index; idx2 = test2.index
    train2.reset_index(drop=True,inplace=True)

    # FEATURE SELECTION (USE APPROX 40 OF 255 FEATURES)
    train3 = sel.transform(train2[cols])
    test3 = sel.transform(test2[cols])

    # STRATIFIED K-FOLD
    skf = StratifiedKFold(n_splits=11, random_state=random_state, shuffle=True)
    for train_index, test_index in skf.split(train3, train2['target']):
        
        gm2.fit(np.concatenate([train3[train_index,:],test3],axis = 0))
        # oof[idx1[test_index]] = gm2.predict_proba(train3[test_index,:])[:,0]
        oof[idx1[test_index]] = np.array([sum(x[CLUSTER_NUM:]) for x in gm2.predict_proba(train3[test_index,:])])
        preds[idx2] += np.array([sum(x[CLUSTER_NUM:]) for x in gm2.predict_proba(test3)]) / skf.n_splits


    """
    X_train_with_c = pipe.fit_transform(X_train_with_c_df[cols])
    gm2.fit(X_train_with_c)
    print('iter: ', gm2.n_iter_)

    oof_cluster = gm2.predict(X_train_with_c)
    plt.subplot(1,2,2)
    plt.hist(oof_cluster)

    print(X_train_with_c_df.cluster.values[:10])
    print(oof_cluster[:10])
    print("cluster accuracy: ",
        len(X_train_with_c_df[X_train_with_c_df['cluster'] == oof_cluster]) / len(X_train_with_c_df))

    oof = np.array([0 if x < CLUSTER_NUM else 1 for x in oof_cluster])
    auc = roc_auc_score(X_train_with_c_df['target'],oof)
    print('GMM scores CV =',round(auc,5))

    clf = QuadraticDiscriminantAnalysis(reg_param=0.5)
    clf.fit(X_train_with_c, X_train_with_c_df['target'])
    oof = clf.predict(X_train_with_c)
    auc = roc_auc_score(X_train_with_c_df['target'],oof)
    print('QDA scores CV =',round(auc,5))
    """

# PRINT CV AUC
auc = roc_auc_score(train['target'],oof)
print('QDA scores CV =',round(auc,5))


In [ ]:
sub = pd.read_csv('../input/sample_submission.csv')
sub['target'] = preds
sub.to_csv('submission.csv', index=False)